# Wine Quality Prediction using Decision Trees

This dataset is related to red variants of the Portuguese "Vinho Verde" wine. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.). These datasets can be viewed as classification or regression tasks. The classes are ordered and not balanced (e.g. there are much more normal wines than excellent or poor ones).

## Problem Statement:
    
We will apply a method of assessing wine quality using a Random Forest Algorithm, with hyperparameter tuning using skopt's baye search.

#### Number of Instances: 
- red wine - 1599; white wine - 4898. 

### Attribute information:
  - Input variables (based on physicochemical tests):
      1. fixed acidity (tartaric acid - g / dm^3)
      2. volatile acidity (acetic acid - g / dm^3)
      3. citric acid (g / dm^3)
      4. residual sugar (g / dm^3)
      5. chlorides (sodium chloride - g / dm^3
      6. free sulfur dioxide (mg / dm^3)
      7. total sulfur dioxide (mg / dm^3)
      8. density (g / cm^3)
      9. pH
      10. sulphates (potassium sulphate - g / dm3)
      11. alcohol (% by volume)
  - Output variable (based on sensory data): 
      - quality (score between 0 and 10)


### Missing Attribute Values: None

### Description of attributes:

   1 - fixed acidity: most acids involved with wine or fixed or nonvolatile (do not evaporate readily)

   2 - volatile acidity: the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste

   3 - citric acid: found in small quantities, citric acid can add 'freshness' and flavor to wines

   4 - residual sugar: the amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet

   5 - chlorides: the amount of salt in the wine

   6 - free sulfur dioxide: the free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine

   7 - total sulfur dioxide: amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2 concentrations over 50 ppm, SO2 becomes evident in the nose and taste of wine

   8 - density: the density of water is close to that of water depending on the percent alcohol and sugar content

   9 - pH: describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale

   10 - sulphates: a wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant

   11 - alcohol: the percent alcohol content of the wine

   Output variable (based on sensory data): 
   12 - quality (score between 0 and 10)


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn
import seaborn as sns
sns.set(style='darkgrid')

data = pd.read_csv('/kaggle/input/wine-quality-dataset/WineQT.csv')
data.head()

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
data.info()

1. No missing values.
2. Id needs to be removed.
3. quality is the output variable.
4. All float64 values. All Real values.
5. Minimal EDA needed.

In [ ]:
#removing Id
data.drop('Id',axis=1,inplace=True)

In [ ]:
data['quality'].value_counts()

6. Highly imbalanced output feature. Using SMOTE-ENN to increase representation. Let's have a look at correlation plot first.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(),
            annot=True,
            linewidths=.5,
            center=0,
            cbar=False,
            cmap="YlGnBu")
plt.show()

In [ ]:
X = data.drop('quality', axis=1)
y = data['quality']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range =(0, 1))
columns = X.columns
scaler.fit(X)

X = pd.DataFrame(scaler.transform(X))
X.columns = columns

In [ ]:
X.head()

In [ ]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

smote_enn = SMOTEENN(random_state=42)
X, y = smote_enn.fit_resample(X, y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=.25, random_state=22)

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, n_jobs=-1)

search_space = {
        "max_depth": Integer(3, 13), 
        "n_estimators": Integer(10, 30)
    }

search = BayesSearchCV(rf, search_space, n_iter=20, scoring="accuracy", cv=5, verbose= 0)
search.fit(X_train, y_train)

In [ ]:
search.best_params_

In [ ]:
rf.set_params(**search.best_params_)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
print(rf.score(X_train, y_train))  # performance on train data)
print(rf.score(X_test, y_test))  # performance on test data)

In [ ]:
y_pred_train = rf.predict(X_train)

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

accuracy = metrics.accuracy_score(y_train, y_pred_train)
print("Accuracy: {:.2f}".format(accuracy))
cm=confusion_matrix(y_train,y_pred_train)
print('Confusion Matrix: \n', cm)
print(classification_report(y_train, y_pred_train))

In [ ]:
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus
import graphviz

xvar = data.drop('quality', axis=1)
feature_cols = xvar.columns

In [ ]:
feature_importances = pd.DataFrame({'features': X_train.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values('Importance',ascending=False).reset_index(drop=True)
feature_importances